In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage
import os
from sklearn.metrics import mean_squared_error
from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means
from skimage.measure import compare_ssim
from sklearn.metrics import mean_squared_error
from skimage.measure import compare_psnr,compare_mse
from random import shuffle
import math
from keras.models import model_from_json, load_model
from keras.layers import *
from keras.models import Model
from keras import regularizers
from keras.utils import plot_model
from keras import backend as K
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger, EarlyStopping, ModelCheckpoint
from keras.layers.core import Lambda
from keras.preprocessing.image import ImageDataGenerator

import pickle
from random import randint
from utilities import *
%matplotlib inline


Using TensorFlow backend.


In [2]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,:,0], rgb[:,:,:,1], rgb[:,:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path,filename),0)
        if img is not None:
            images.append(img)
    return np.array(images)
        

In [3]:
from PIL import Image

In [3]:
def resize_images(src_path,dest_path,sizeX,sizeY):
    images = []
    width,height = sizeX,sizeY
    for filename in os.listdir(src_path):
        img = cv2.imread(src_path + filename)
        if img is not None:
            images.append(img)
    cnt =1
    for img in images:
        im2 = cv2.resize(img,(width, height),interpolation = cv2.INTER_CUBIC)
        
        
        ext = ".png"
        cv2.imwrite(dest_path + "image" +  str(cnt) + ext,im2)
        cnt+=1

In [5]:
raw_images = read_images_from_folder('./data/oct_gt/')
raw_images = np.resize(raw_images,(raw_images.shape[0],448,900,1))

In [4]:
#Adding noise to images
def add_noise(images):
    
    noisy_set = []
    for image in images:
        print len(noisy_set)
        for shape in xrange(2,9,1):
            for scale in xrange(2,9,1):
                noise = np.random.gamma(shape,scale,image.shape)
                noisy_set.append((image,np.clip(noise + image,0.,255.)))
    shuffle(noisy_set)
    return np.array(noisy_set)
   
#Shuffle the noisy image ground truth pair to randomize the noise distribution in the dataset
def expand_pair(noisy_set):   
    ground_truth=[]
    noisy_images = []
    for i in range(noisy_set.shape[0]):
        ground_truth.append(noisy_set[i][0].reshape((noisy_set[i][0].shape[0],noisy_set[i][0].shape[1],1)))
        #print( str(noisy_set[i][0].shape[0]) +" "+ str(noisy_set[i][0].shape[1]))
        noisy_images.append(noisy_set[i][1].reshape((noisy_set[i][1].shape[0],noisy_set[i][1].shape[1],1)))
    return np.array(ground_truth), np.array(noisy_images)

In [7]:
noisy_set = add_noise(raw_images)
for i in range(noisy_set.shape[0]):
    cv2.imwrite("./data/oct_noisy/"+str(i)+".tif",noisy_set[i][1])    

0
49
98
147
196
245
294
343
392
441
490
539
588
637
686
735
784


In [ ]:
#Shuffling and adding noise to the dataset
ground_truth,noisy_images = expand_pair(noisy_set)
print (ground_truth.shape, noisy_images.shape, noisy_set.shape)

((833, 448, 900, 1), (833, 448, 900, 1), (833, 2, 448, 900, 1))


In [ ]:
print np.std(ground_truth/255.)

In [5]:
def resize_images(images,size):
    resized_images = []
    
    for img in images:
        im2 = cv2.resize(img,size,interpolation = cv2.INTER_CUBIC)
        resized_images.append(im2)
    
    resized_images = np.array(resized_images)
    resized_images = np.reshape(resized_images,(resized_images.shape[0],resized_images.shape[1],resized_images.shape[2],1))
    
    return resized_images

ground_truth_rs = resize_images(ground_truth,(256,128))
noisy_images_rs = resize_images(noisy_images,(256,128))

NameError: name 'ground_truth' is not defined

In [11]:
noisy_images_rs.shape

(833, 128, 256, 1)

In [12]:
for i in range(noisy_images_rs.shape[0]):
    cv2.imwrite("./data/oct_noisy_resized/"+str(i)+".tif",noisy_images_rs[i])  
    
for i in range(ground_truth_rs.shape[0]):
    cv2.imwrite("./data/oct_gt_resized/"+str(i)+".tif",ground_truth_rs[i])    

In [6]:
ground_truth_rs = read_images_from_folder('./data/oct_gt_resized/')
ground_truth_rs = np.resize(ground_truth_rs,(ground_truth_rs.shape[0],ground_truth_rs.shape[1],ground_truth_rs.shape[2],1))

noisy_images_rs = read_images_from_folder('./data/oct_noisy_resized/')
noisy_images_rs = np.resize(noisy_images_rs,(noisy_images_rs.shape[0],noisy_images_rs.shape[1],noisy_images_rs.shape[2],1))

In [7]:
cv2.imwrite('test_noisy.png',noisy_images_rs[33])
cv2.imwrite('test.png',ground_truth_rs[33])

True

In [8]:
#Split into training and cross validation and normalizing
train_size = int(ground_truth_rs.shape[0]*0.8)
x_train = ground_truth_rs[0:train_size]/255.
x_train_noisy = noisy_images_rs[0:train_size]/255.
x_test = ground_truth_rs[train_size:]/255.
x_test_noisy = noisy_images_rs[train_size:]/255.
print (x_train_noisy.shape)
print (x_test_noisy.shape)

(666, 128, 256, 1)
(167, 128, 256, 1)


In [32]:
#for i in range(ground_truth.shape[0]):
 #   cv2.imwrite("./data/cifar-bw/image"+str(i)+".png",ground_truth[i])
    
#for i in range(noisy_images.shape[0]):
 #   cv2.imwrite("./data/noisy-cifar-bw/image"+str(i)+".png",noisy_images[i])

In [10]:
#lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
#csv_logger = CSVLogger('./models/simple_cnn_autoencoder.csv')
#early_stopper = EarlyStopping(min_delta=0.001,patience=30)
model_checkpoint = ModelCheckpoint('./models/simple_cnn_oct.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [9]:
x_train_noisy.shape

(666, 128, 256, 1)

In [35]:
#Defining the model

def get_simple_cnn_autoencoder_model(model_path=None):
    
    if(model_path is None):
        autoencoder = None
    else:
        autoencoder = read_model_json(model_path) 
    
    if(autoencoder is None):
        input_img = Input(shape=((None,None,1)))  # adapt this if using `channels_first` image data format

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = AveragePooling2D((2, 2), padding='same')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        encoded = AveragePooling2D((2, 2), padding='same')(x)

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        
        decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

        autoencoder = Model(input_img, decoded)
        
        
        autoencoder.compile(optimizer='Adam', loss='binary_crossentropy',metrics = ['accuracy','mean_squared_error'])

    print (autoencoder.summary())
    return autoencoder

In [36]:
#Training the model
autoencoder = get_simple_cnn_autoencoder_model()
plot_model(autoencoder, to_file='model.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, None, None, 64)    640       
_________________________________________________________________
average_pooling2d_3 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
average_pooling2d_4 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, None, None, 64)    0         
__________

In [39]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=500,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder', histogram_freq=0, write_graph=True), model_checkpoint])

Train on 666 samples, validate on 167 samples
Epoch 1/500
666/666 [==============================] - 53s - loss: 0.6267 - acc: 0.0000e+00 - mean_squared_error: 0.0120 - val_loss: 0.6170 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0063
Epoch 2/500
666/666 [==============================] - 48s - loss: 0.6117 - acc: 0.0000e+00 - mean_squared_error: 0.0052 - val_loss: 0.6140 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0049
Epoch 3/500
666/666 [==============================] - 48s - loss: 0.6090 - acc: 0.0000e+00 - mean_squared_error: 0.0039 - val_loss: 0.6176 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0065
Epoch 4/500
666/666 [==============================] - 48s - loss: 0.6071 - acc: 0.0000e+00 - mean_squared_error: 0.0030 - val_loss: 0.6149 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0053
Epoch 5/500
666/666 [==============================] - 48s - loss: 0.6061 - acc: 0.0000e+00 - mean_squared_error: 0.0026 - val_loss: 0.6094 - val_acc: 0.0000e+00 - val_mean_s

666/666 [==============================] - 48s - loss: 0.6033 - acc: 0.0000e+00 - mean_squared_error: 0.0014 - val_loss: 0.6075 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0019
Epoch 23/500
666/666 [==============================] - 48s - loss: 0.6035 - acc: 0.0000e+00 - mean_squared_error: 0.0014 - val_loss: 0.6066 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0015
Epoch 24/500
666/666 [==============================] - 48s - loss: 0.6033 - acc: 0.0000e+00 - mean_squared_error: 0.0014 - val_loss: 0.6067 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0015
Epoch 25/500
666/666 [==============================] - 48s - loss: 0.6032 - acc: 0.0000e+00 - mean_squared_error: 0.0013 - val_loss: 0.6060 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0013
Epoch 26/500
666/666 [==============================] - 48s - loss: 0.6030 - acc: 0.0000e+00 - mean_squared_error: 0.0012 - val_loss: 0.6060 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0012
Epoch 27/500
666/666 [===========

666/666 [==============================] - 48s - loss: 0.6028 - acc: 0.0000e+00 - mean_squared_error: 0.0011 - val_loss: 0.6058 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0011
Epoch 45/500
666/666 [==============================] - 48s - loss: 0.6029 - acc: 0.0000e+00 - mean_squared_error: 0.0011 - val_loss: 0.6060 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0012
Epoch 46/500
666/666 [==============================] - 48s - loss: 0.6030 - acc: 0.0000e+00 - mean_squared_error: 0.0012 - val_loss: 0.6058 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0012
Epoch 47/500
666/666 [==============================] - 48s - loss: 0.6030 - acc: 0.0000e+00 - mean_squared_error: 0.0012 - val_loss: 0.6058 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0011
Epoch 48/500
666/666 [==============================] - 48s - loss: 0.6031 - acc: 0.0000e+00 - mean_squared_error: 0.0012 - val_loss: 0.6063 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0014
Epoch 49/500
666/666 [===========

666/666 [==============================] - 48s - loss: 0.6025 - acc: 0.0000e+00 - mean_squared_error: 9.7271e-04 - val_loss: 0.6055 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0010
Epoch 67/500
666/666 [==============================] - 48s - loss: 0.6027 - acc: 0.0000e+00 - mean_squared_error: 0.0011 - val_loss: 0.6054 - val_acc: 0.0000e+00 - val_mean_squared_error: 9.6456e-04
Epoch 68/500
666/666 [==============================] - 48s - loss: 0.6024 - acc: 0.0000e+00 - mean_squared_error: 9.2018e-04 - val_loss: 0.6056 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0010
Epoch 69/500
666/666 [==============================] - 48s - loss: 0.6024 - acc: 0.0000e+00 - mean_squared_error: 9.4691e-04 - val_loss: 0.6058 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0011
Epoch 70/500
666/666 [==============================] - 48s - loss: 0.6025 - acc: 0.0000e+00 - mean_squared_error: 9.6363e-04 - val_loss: 0.6055 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0010
Epoch 71/500


666/666 [==============================] - 48s - loss: 0.6023 - acc: 0.0000e+00 - mean_squared_error: 8.7892e-04 - val_loss: 0.6051 - val_acc: 0.0000e+00 - val_mean_squared_error: 8.3497e-04
Epoch 89/500
666/666 [==============================] - 48s - loss: 0.6021 - acc: 0.0000e+00 - mean_squared_error: 8.1749e-04 - val_loss: 0.6051 - val_acc: 0.0000e+00 - val_mean_squared_error: 8.5429e-04
Epoch 90/500
666/666 [==============================] - 48s - loss: 0.6021 - acc: 0.0000e+00 - mean_squared_error: 8.1012e-04 - val_loss: 0.6051 - val_acc: 0.0000e+00 - val_mean_squared_error: 8.3997e-04
Epoch 91/500
666/666 [==============================] - 48s - loss: 0.6022 - acc: 0.0000e+00 - mean_squared_error: 8.6174e-04 - val_loss: 0.6053 - val_acc: 0.0000e+00 - val_mean_squared_error: 9.1487e-04
Epoch 92/500
666/666 [==============================] - 48s - loss: 0.6022 - acc: 0.0000e+00 - mean_squared_error: 8.3939e-04 - val_loss: 0.6051 - val_acc: 0.0000e+00 - val_mean_squared_error: 8.58

...

666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.3690e-04 - val_loss: 0.6048 - val_acc: 0.0000e+00 - val_mean_squared_error: 7.1137e-04
Epoch 403/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.2100e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.2862e-04
Epoch 404/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.2494e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.3283e-04
Epoch 405/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 6.0190e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.1692e-04
Epoch 406/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 6.0027e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 

666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 5.9804e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.2636e-04
Epoch 425/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.1378e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.3622e-04
Epoch 426/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 6.0927e-04 - val_loss: 0.6047 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.6385e-04
Epoch 427/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.1844e-04 - val_loss: 0.6047 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.7486e-04
Epoch 428/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.3352e-04 - val_loss: 0.6051 - val_acc: 0.0000e+00 - val_mean_squared_error: 

666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 5.9780e-04 - val_loss: 0.6047 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.8982e-04
Epoch 448/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 6.0615e-04 - val_loss: 0.6049 - val_acc: 0.0000e+00 - val_mean_squared_error: 7.5683e-04
Epoch 449/500
666/666 [==============================] - 48s - loss: 0.6018 - acc: 0.0000e+00 - mean_squared_error: 6.7082e-04 - val_loss: 0.6049 - val_acc: 0.0000e+00 - val_mean_squared_error: 7.4135e-04
Epoch 450/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.1743e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.3384e-04
Epoch 451/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.2551e-04 - val_loss: 0.6049 - val_acc: 0.0000e+00 - val_mean_squared_error: 

666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.5207e-04 - val_loss: 0.6047 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.8448e-04
Epoch 470/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 5.8765e-04 - val_loss: 0.6047 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.7465e-04
Epoch 471/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 6.0559e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.3203e-04
Epoch 472/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 5.9580e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.2648e-04
Epoch 473/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.2207e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 

666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 6.0320e-04 - val_loss: 0.6045 - val_acc: 0.0000e+00 - val_mean_squared_error: 5.9663e-04
Epoch 493/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 5.7459e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.2028e-04
Epoch 494/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.2781e-04 - val_loss: 0.6047 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.7466e-04
Epoch 495/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 5.9491e-04 - val_loss: 0.6045 - val_acc: 0.0000e+00 - val_mean_squared_error: 5.9256e-04
Epoch 496/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 5.8406e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 

In [12]:

#Function to get saved keras model
def read_model_json(jsonfilePath,h5filePath):
    try:
        json_file = open(jsonfilePath, 'r')
        print json_file
        loaded_model_json = json_file.read()
        json_file.close()
        print "hello"
        loaded_model = model_from_json(loaded_model_json)
         
        # load weights into new model
        loaded_model.load_weights(h5filePath)

        return loaded_model
    except:
        return None

In [10]:
def get_gated_connections(gatePercentageFactor,inputLayer):
    gateFactor = Input(tensor = K.variable([gatePercentageFactor]))
    fractionG = Lambda(lambda x: x[0]*x[1])([inputLayer,gateFactor])
    complement = Lambda(lambda x: x[0] - x[1])([inputLayer,fractionG])
    
    return gateFactor,fractionG,complement

#x is conv layer
#y is de-conv layer
#gf is gating factor
#fg is fractional input from gate
#c is complement ie remaining fraction from the gate
#jt joining tensor of convolution layer and previous de-conv layer 

def get_cnn_dsc_architecture(model_path=None):
    
    if(model_path is None):
        sym_autoencoder = None
    else:
        sym_autoencoder = read_model_json(model_path[0],model_path[1])
        print model_path[0],model_path[1]
    if(sym_autoencoder is None):
        input_img = Input(shape=(None,None,1), name = "Image_input")  # adapt this if using `channels_first` image data format
        x1 = Conv2D(64, (4, 4), activation='relu', padding='same')(input_img)
        gf1,fg1,c1 = get_gated_connections(0.1,x1)

        x = MaxPooling2D((2, 2), padding='same')(fg1)
        x2 = Conv2D(64, (4, 4), activation='relu', padding='same')(x) 
        gf2,fg2,c2 = get_gated_connections(0.2,x2)

        x = MaxPooling2D((2, 2), padding='same')(fg2)
        x3 = Conv2D(128, (4, 4), activation='relu', padding='same')(x) 
        gf3,fg3,c3 = get_gated_connections(0.3,x3)

        x = MaxPooling2D((2, 2), padding='same')(x3)
        x4 = Conv2D(256, (4, 4), activation='relu', padding='same')(x) 
        gf4,fg4,c4 = get_gated_connections(0.4,x4)

        x = MaxPooling2D((2, 2), padding='same')(x4)
        x5 = Conv2D(512, (4, 4), activation='relu', padding='same')(x) 

        x = UpSampling2D((2, 2))(x5)
        y1 = Conv2DTranspose(256, (4, 4), activation='relu', padding='same')(x) 
        jt4 = Add()([y1,c4])
        x = UpSampling2D((2, 2))(jt4)

        y2 = Conv2DTranspose(128, (4, 4), activation='relu', padding='same')(x) 
        jt3 = Add()([y2,c3])
        x = UpSampling2D((2, 2))(jt3)

        y3 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(x) 
        jt2 = Add()([y3,c2])
        x = UpSampling2D((2, 2))(jt2)

        jt1 = Add()([x,c1])
        y4 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(jt1)
        y5 = Conv2DTranspose(1, (4, 4), activation='relu', padding='same')(y4) 

        layers = y5

        sym_autoencoder = Model([input_img,gf1,gf2,gf3,gf4],layers)
        sym_autoencoder.compile(optimizer='sgd', loss = 'mean_squared_error', metrics = ['accuracy','mean_squared_error'])
        print "Model created"
    else:
        print "Saved model loaded"
    print sym_autoencoder.summary()
    return sym_autoencoder

In [11]:
sym_autoencoder = get_cnn_dsc_architecture()
plot_model(autoencoder, to_file='sys.png')

Model created
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Image_input (InputLayer)         (None, None, None, 1) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 64 1088        Image_input[0][0]                
____________________________________________________________________________________________________
input_1 (InputLayer)             (1,)                  0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, None, None, 64 0           conv2d_1[0][0]                   
                                                                   input_1[0]

NameError: name 'autoencoder' is not defined

In [12]:
model_checkpoint1 = ModelCheckpoint('./models/gated_cnn_autoencoder_oct.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [ ]:
sym_autoencoder.fit(x_train_noisy, x_train,
                epochs=500,
                batch_size=8,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/gated_cnn_autoencoder', 
                                       histogram_freq=0,
                                       write_graph=True),model_checkpoint1])

Train on 666 samples, validate on 167 samples
Epoch 1/500
666/666 [==============================] - 181s - loss: 0.0351 - acc: 0.0000e+00 - mean_squared_error: 0.0351 - val_loss: 0.0064 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0064
Epoch 2/500
666/666 [==============================] - 168s - loss: 0.0055 - acc: 0.0000e+00 - mean_squared_error: 0.0055 - val_loss: 0.0058 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0058
Epoch 3/500
666/666 [==============================] - 168s - loss: 0.0051 - acc: 0.0000e+00 - mean_squared_error: 0.0051 - val_loss: 0.0056 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0056
Epoch 4/500
666/666 [==============================] - 168s - loss: 0.0048 - acc: 0.0000e+00 - mean_squared_error: 0.0048 - val_loss: 0.0053 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0053
Epoch 5/500
666/666 [==============================] - 168s - loss: 0.0046 - acc: 0.0000e+00 - mean_squared_error: 0.0046 - val_loss: 0.0055 - val_acc: 0.0000e+00 - val_m

666/666 [==============================] - 168s - loss: 0.0037 - acc: 0.0000e+00 - mean_squared_error: 0.0037 - val_loss: 0.0046 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0046
Epoch 21/500
666/666 [==============================] - 168s - loss: 0.0037 - acc: 0.0000e+00 - mean_squared_error: 0.0037 - val_loss: 0.0044 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0044
Epoch 22/500
666/666 [==============================] - 168s - loss: 0.0036 - acc: 0.0000e+00 - mean_squared_error: 0.0036 - val_loss: 0.0049 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0049
Epoch 23/500
666/666 [==============================] - 168s - loss: 0.0036 - acc: 0.0000e+00 - mean_squared_error: 0.0036 - val_loss: 0.0042 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0042
Epoch 24/500
666/666 [==============================] - 168s - loss: 0.0036 - acc: 0.0000e+00 - mean_squared_error: 0.0036 - val_loss: 0.0042 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0042
Epoch 25/500
666/666 [======

666/666 [==============================] - 168s - loss: 0.0031 - acc: 0.0000e+00 - mean_squared_error: 0.0031 - val_loss: 0.0038 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0038
Epoch 41/500
666/666 [==============================] - 168s - loss: 0.0031 - acc: 0.0000e+00 - mean_squared_error: 0.0031 - val_loss: 0.0039 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0039
Epoch 42/500
666/666 [==============================] - 168s - loss: 0.0031 - acc: 0.0000e+00 - mean_squared_error: 0.0031 - val_loss: 0.0039 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0039
Epoch 43/500
666/666 [==============================] - 168s - loss: 0.0031 - acc: 0.0000e+00 - mean_squared_error: 0.0031 - val_loss: 0.0036 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0036
Epoch 44/500
666/666 [==============================] - 168s - loss: 0.0030 - acc: 0.0000e+00 - mean_squared_error: 0.0030 - val_loss: 0.0036 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0036
Epoch 45/500
666/666 [======

666/666 [==============================] - 168s - loss: 0.0027 - acc: 0.0000e+00 - mean_squared_error: 0.0027 - val_loss: 0.0039 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0039
Epoch 62/500
666/666 [==============================] - 168s - loss: 0.0028 - acc: 0.0000e+00 - mean_squared_error: 0.0028 - val_loss: 0.0033 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0033
Epoch 63/500
666/666 [==============================] - 168s - loss: 0.0027 - acc: 0.0000e+00 - mean_squared_error: 0.0027 - val_loss: 0.0036 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0036
Epoch 64/500
666/666 [==============================] - 168s - loss: 0.0027 - acc: 0.0000e+00 - mean_squared_error: 0.0027 - val_loss: 0.0033 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0033
Epoch 65/500
666/666 [==============================] - 168s - loss: 0.0027 - acc: 0.0000e+00 - mean_squared_error: 0.0027 - val_loss: 0.0033 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0033
Epoch 66/500
666/666 [======

666/666 [==============================] - 168s - loss: 0.0025 - acc: 0.0000e+00 - mean_squared_error: 0.0025 - val_loss: 0.0030 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0030
Epoch 82/500
666/666 [==============================] - 168s - loss: 0.0025 - acc: 0.0000e+00 - mean_squared_error: 0.0025 - val_loss: 0.0030 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0030
Epoch 83/500
666/666 [==============================] - 168s - loss: 0.0025 - acc: 0.0000e+00 - mean_squared_error: 0.0025 - val_loss: 0.0036 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0036
Epoch 84/500
666/666 [==============================] - 168s - loss: 0.0025 - acc: 0.0000e+00 - mean_squared_error: 0.0025 - val_loss: 0.0029 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0029
Epoch 85/500
666/666 [==============================] - 168s - loss: 0.0025 - acc: 0.0000e+00 - mean_squared_error: 0.0025 - val_loss: 0.0032 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0032
Epoch 86/500
666/666 [======

...

666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0019 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0019
Epoch 330/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0019 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0019
Epoch 331/500
666/666 [==============================] - 168s - loss: 0.0018 - acc: 0.0000e+00 - mean_squared_error: 0.0018 - val_loss: 0.0029 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0029
Epoch 332/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0022 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0022
Epoch 333/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0025 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0025
Epoch 334/500
666/666 [=

666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0018 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0018
Epoch 354/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0019 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0019
Epoch 355/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0027 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0027
Epoch 356/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0024 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0024
Epoch 357/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0019 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0019
Epoch 358/500
666/666 [=

666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0018 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0018
Epoch 377/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0024 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0024
Epoch 378/500
666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0021 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0021
Epoch 379/500
666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0024 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0024
Epoch 380/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0024 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0024
Epoch 381/500
666/666 [=

666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0028 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0028
Epoch 400/500
666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0022 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0022
Epoch 401/500
666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0018 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0018
Epoch 402/500
666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0024 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0024
Epoch 403/500
666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0022 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0022
Epoch 404/500
666/666 [=

168/666 [======>.......................] - ETA: 117s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017

In [21]:
#getting testing images

In [28]:
autoencoder = load_model('./models/simple_cnn_autoencoder.hdf5')

In [101]:
#plt.imshow(out_autoencoder[0])

def predict_save(model,image_path):
    test = cv2.imread(image_path,0)
    test = np.array(test)
    test = test.reshape(1,test.shape[0],test.shape[1],1)
    test =  test/255.0
    
    out = model.predict(test,verbose=1)
    cv2.imwrite("out.png",(out[0]*255))


In [103]:
predict_save(autoencoder,'./data/standard_test_images/noise/image21.png')

1/1 [==============================] - 0s


In [119]:
for i in range(100):
    cv2.imwrite("./data/denoise-cifar-bw/"+str(i)+".png",out_autoencoder[i]*255.)

In [6]:

def get_psnr(imageA,imageB):
    maxI = 1.0
    try:
        return 20*math.log10(maxI) - 10*math.log10(compare_mse(imageA.flatten(),imageB.flatten()))
    except:
        return 20*math.log10(maxI)

def get_psnr_result(x_test, out):
    psnr_sum = 0
    for i in range(out.shape[0]):
        psnr_sum += compare_psnr(x_test[i].reshape(32,32,1),out[i].reshape(32,32,1),data_range=255)
        
    return 1.0*psnr_sum/out.shape[0];

def get_ssim_result(originalSet,noisySet):
    ssim_sum = 0
    originalSet = originalSet.reshape(originalSet.shape[0],32,32,1)
    noisySet = noisySet.reshape(noisySet.shape[0],32,32,1)
    for i in range(originalSet.shape[0]):
        ssim_sum += ssim(originalSet[i], noisySet[i],data_range=originalSet[i].max() - noisySet[i].min(), multichannel=True)
    return 1.0*ssim_sum/originalSet.shape[0]

In [7]:
def bm3d_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],32,32)
    denoised = []
    count = 1
    for i in range(noisy_image.shape[0]):
        Basic_img = bm3d.BM3D_1st_step(noisy_image[i])
        Final_img = bm3d.BM3D_2nd_step(Basic_img, noisy_image[i])
        denoised.append(Final_img)
        if (count%10 == 0):
            print (str(count)+ "images denoised")
        count+=1
        
    return np.array(denoised)

def nlm_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],32,32,1)
    denoised = []
    count = 1
    
    for image in noisy_image:
        denoised_image = denoise_nl_means(image, 7, 11, 0.5,multichannel = False)
        denoised.append(denoised_image)
        if(count%100 == 0) :
            print(str(count)+" images denoised")
        count+=1
    return np.array(denoised)

In [134]:
print (test_images.shape,out_autoencoder.shape)
print (compare_mse(test_images[0].flatten(),(out_autoencoder[0]*255.).flatten()))
get_psnr_result(out_autoencoder*255.,test_images)

((10000, 32, 32, 1), (10000, 32, 32, 1))
458.092258419


22.066576979897317

In [39]:
print (test_images.shape,out_sym_autoencoder.shape)
print (mean_squared_error(test_images[0].flatten(),out_sym_autoencoder[0].flatten()))
get_psnr_result(out_sym_autoencoder,test_images)

NameError: name 'out_sym_autoencoder' is not defined

In [39]:
import bm3d

In [40]:
noisy_test_images.shape
bm3d_out = bm3d_denoise(noisy_test_images)

error: /home/travis/miniconda/conda-bld/work/opencv-2.4.11/modules/core/src/dxt.cpp:2330: error: (-213) Odd-size DCT's are not implemented in function dct


In [ ]:
bm3d_out_norm = bm3d_out.astype('float64')/255.0

In [ ]:
get_psnr_result(bm3d_out_norm,test_images)

In [166]:
print(noisy_test_images.shape)
nlm_out = nlm_denoise(noisy_test_images*255.)
nlm_out = nlm_out.astype('float64')

(10000, 32, 32, 1)


KeyboardInterrupt: 

In [157]:
print nlm_out.shape,test_images.shape
get_psnr_result(nlm_out,test_images)

(10000, 32, 32) (10000, 32, 32, 1)


18.414818545893922

In [136]:
get_ssim_result(test_images,out_autoencoder*255.)

0.69608629344603479

In [66]:
get_ssim_result(test_images,out_sym_autoencoder)

0.718734871276698

In [ ]:
get_ssim_result(test_images,bm3d_out_norm)

In [69]:
get_ssim_result(test_images,nlm_out)

0.49438484841716263

In [22]:
model = load_model("./models/oct_epoch_500_batch_20/simple_cnn_oct.hdf5")

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    640       
_________________________________________________________________
average_pooling2d_1 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
average_pooling2d_2 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, None, None, 64)    0         
__________

In [ ]:
#Testing


In [23]:
def read_images_from_folder_with_names(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path,filename),0)
        if img is not None:
            images.append((filename,img))
    return images
       

In [28]:
#Adding Gamma noise
test_image = cv2.imread('./test_data/5.tif',0)
test_image = cv2.resize(test_image,(900,448),interpolation = cv2.INTER_CUBIC)
print test_image.shape
gamma_param = [(3,2),(4,3),(4,4),(6,6),(8,7)]
for param in gamma_param:
    shape = param[0]
    scale = param[1]
    noise = np.random.gamma(shape,scale,test_image.shape)
    noisy_img = np.clip(noise + test_image,0.,255.)
    cv2.imwrite('./test_data/noisy/oct_shape_'+str(shape)+'_scale_'+str(scale)+'.tif',noisy_img)

(448, 900)


In [29]:
test_image_set = read_images_from_folder_with_names('./test_data/noisy/')

In [30]:
test_image_set

[('oct_shape_8_scale_7.tif', array([[179,  93, 157, ..., 124, 137, 138],
         [150, 115, 103, ..., 134, 118, 149],
         [156,  86, 123, ..., 104, 109, 109],
         ..., 
         [126,  69,  81, ..., 119, 110,  83],
         [ 74,  77,  78, ..., 108,  80, 101],
         [144,  79, 137, ..., 117, 113,  87]], dtype=uint8)),
 ('oct_shape_6_scale_6.tif', array([[136, 110,  90, ...,  87,  85,  97],
         [ 98,  91, 162, ...,  95, 103, 126],
         [141, 105, 111, ..., 120,  94, 103],
         ..., 
         [ 78, 102,  84, ...,  88,  61,  85],
         [ 71,  81,  74, ...,  80,  86,  80],
         [ 78,  86, 107, ...,  92, 100,  62]], dtype=uint8)),
 ('oct_shape_4_scale_3.tif', array([[ 84,  76,  88, ...,  62,  86,  93],
         [ 77,  92, 101, ...,  79,  76,  84],
         [100,  71,  97, ...,  85,  86,  78],
         ..., 
         [ 41,  63,  62, ...,  49,  61,  56],
         [ 53,  47,  64, ...,  47,  72,  55],
         [ 54,  48,  48, ...,  65,  58,  59]], dtype=uint8))

In [31]:
for idx,test_image in enumerate(test_image_set):
    #test_image_norm = cv2.resize(test_image,(256,128),interpolation = cv2.INTER_CUBIC)
    test_image_norm = [test_image[1]]
    test_image_norm = np.array(test_image_norm,dtype=np.float32)
    test_image_norm = test_image_norm/255.
    test_image_norm = test_image_norm.reshape((test_image_norm.shape[0],test_image_norm.shape[1],test_image_norm.shape[2],1))
    
    out_image = model.predict(test_image_norm,verbose=1)
    print test_image_norm[0].shape,out_image[0].shape
    psnr = compare_psnr(test_image_norm[0]*255.,out_image[0]*255.,data_range=256)
    ssim = compare_ssim(test_image_norm[0],out_image[0],multichannel=True)
    print psnr,ssim
    print test_image[0][:-3]
    cv2.imwrite('./test_data/denoised/'+test_image[0][:-3] + '_'+str(psnr)+'.tif',out_image[0]*255.)
    print test_image_norm.shape

1/1 [==============================] - 2s
(448, 900, 1) (448, 900, 1)
12.0867435625 0.362787193898
oct_shape_8_scale_7.
(1, 448, 900, 1)
1/1 [==============================] - 0s
(448, 900, 1) (448, 900, 1)
15.4330490672 0.506303201293
oct_shape_6_scale_6.
(1, 448, 900, 1)
1/1 [==============================] - 0s
(448, 900, 1) (448, 900, 1)
23.9350194443 0.82419290964
oct_shape_4_scale_3.
(1, 448, 900, 1)
1/1 [==============================] - 0s
(448, 900, 1) (448, 900, 1)
21.6362136498 0.753850963892
oct_shape_4_scale_4.
(1, 448, 900, 1)
1/1 [==============================] - 0s
(448, 900, 1) (448, 900, 1)
27.3646630162 0.891757839356
oct_shape_3_scale_2.
(1, 448, 900, 1)


In [83]:
out_image = model.predict(test_image_norm,verbose=1)

1/1 [==============================] - 0s


In [61]:
cv2.imwrite('denoise_test.tif',out_image[0]*255.)

True

In [24]:
test_image.shape

(450, 900)

In [41]:
model = autoencoder